# Examples from the SICM book

The chapters and subsections correspond to the second edition of SICM. The plots are not optimal yet, for improvement I need to dive more into `thi.ng` or, when eventually available, `plot.ly`. Comments please to https://github.com/kloimhardt

In [1]:
(require '[clojupyter.misc.helper :as helper])
(run! helper/add-dependencies '[[net.littleredcomputer/sicmutils "0.12.0-SNAPSHOT"]
                                [thi.ng/geom "0.0.908"]])
(ns book-examples
    (:refer-clojure :exclude [partial zero? + - * / ref])
    (:require [sicmutils.env :refer :all]
              [sicmutils.mechanics.lagrange :as lg]
              [sicmutils.numerical.minimize :as mn]
              [clojupyter.misc.display :as display]
              [thi.ng.geom.viz.core :as viz]
              [thi.ng.geom.svg.core :as svg]))
(defn tex [x] (-> x tex$$ display/latex))

#'book-examples/tex

## 1.4 Computing Actions

Calculate the action for the free particle along a path. Consider the particle moving at uniform speed along a straight line.

In [2]:
(defn test-path
  "See p. 20"
  [t]
  (up (+ (* 4 t) 7)
      (+ (* 3 t) 5)
      (+ (* 2 t) 1)))

(Lagrangian-action (lg/L-free-particle 3.0) test-path 0.0 10.0)

435.0

#### Paths of minimum Action

Show that the action is smaller along a straight-line test path than along nearby paths

In [3]:
(defn make-η
  "See p. 21"
  [ν t1 t2]
  (fn [t]
    (* (- t t1) (- t t2) (ν t))))

(defn varied-free-particle-action
  "See p. 21"
  [mass q ν t1 t2]
  (fn [ε]
    (let [η (make-η ν t1 t2)]
      (Lagrangian-action (lg/L-free-particle mass)
                         (+ q (* ε η)) t1 t2))))
             
((varied-free-particle-action 3.0 test-path (up sin cos square) 0.0 10.0) 0.01)

564.1214285996772

Simulate lots of the paths in this manner. Proof that the minimum value of the action is the action along the straight path. For this proof it suffices that some optimization parameter is close to zero (need not be exactly zero).

In [4]:
(minimize (varied-free-particle-action 3.0 test-path (up sin cos square) 0.0 10.0) -2 1)

[-9.163058228844889E-10 435.0000000594215 6]

I provide some helper functions for visualization:

In [5]:
(defn points->plot [paths idx1 idx2]
    (letfn [(f1 [v i j] [(nth v i) (nth v j)])
            (f2 [v i j] (mapv #(f1 % i j) v))
            (f3 [p i j] (mapv #(f2 (second %) i j) p))
            (f4 [points stroke]
                {:values  points
                 :attribs {:stroke stroke :stroke-width "2pt"}
                 :layout  viz/svg-scatter-plot})
            (f6 [p] (mapv first p))
            (f7 [p i j] (mapv #(vector %1 %2) (f3 p i j) (f6 p)))
            (f8 [p i j] (mapv #(apply f4 %) (f7 p i j)))
            (f10 [p i] (flatten (f3 p i i)))
            (f11 [p i] (let [d (f10 p i)] [(apply min d) (apply max d)]))]
        
    {:x-axis (viz/linear-axis
              {:domain (f11 paths idx1)
               :range  [50 (- 985 10)]
               :major 1
               :pos    550})
     :y-axis (viz/linear-axis
              {:domain      (f11 paths idx2)
               :range       [550 20]
               :major       1
               :minor       0.2
               :pos         50
               :label-dist  15
               :label-style {:text-anchor "end"}})
     :grid   {:attribs {:stroke "#caa"}
              :minor-x true
              :minor-y false}
     :data (f8 paths idx1 idx2)}))

(defn points-xy->plot [paths] 
  (points->plot paths 1 2))

(defn points-xz->plot [paths] 
  (points->plot paths 1 3))

(defn points-tz->plot [paths] 
  (points->plot paths 0 3))

(defn export-viz
  [spec]
  (->> spec
       (viz/svg-plot2d-cartesian)
       (svg/svg {:width 985 :height 600})
       (svg/serialize)))

(defn alt-range [t0 t1 nofsteps]
  (map float
   (-> (linear-interpolants t0 t1 nofsteps)
     (conj t0)
     vec
     (conj t1))))

(defn make-path-txyz [fn_t t0 t1 nofsteps]
  (mapv #(cons % (vec (rest (simplify (fn_t %)))))
    (alt-range t0 t1 nofsteps)))

(def blue "#0af")
(def orange "#f60")
(def red "#f00")

(comment 
 (def werte {blue [[0 7 5 1] [1 11 8 10]]
            orange [[2 9 2 4] [3 3  9 7]]})

 (-> (points-xy->plot werte)
    export-viz
    display/make-html))

Create data to plot two straight paths in the xy plane. One path is along the x axis (name: path-along-x), the second path leads in all directions.

In [6]:
(defn path-along-x
  [t]
  (up (+ (* 5 t) 1)
      (* 0 t)
      (* 0 t)))

(def path-points-1 {blue (make-path-txyz path-along-x 0 10 50)
                    orange (make-path-txyz test-path 0 10 50)})

#'book-examples/path-points-1

Plot the two paths

In [7]:
(-> (points-xy->plot path-points-1)
    export-viz
    display/make-html)

<?xml version="1.0"?>
 1.00 2.00 3.00 4.00 5.00 6.00 7.00 8.00 9.00 10.00 11.00 12.00 13.00 14.00 15.00 16.00 17.00 18.00 19.00 20.00 21.00 22.00 23.00 24.00 25.00 26.00 27.00 28.00 29.00 30.00 31.00 32.00 33.00 34.00 35.00 36.00 37.00 38.00 39.00 40.00 41.00 42.00 43.00 44.00 45.00 46.00 47.00 48.00 49.00 50.00 51.00 0.00 1.00 2.00 3.00 4.00 5.00 6.00 7.00 8.00 9.00 10.00 11.00 12.00 13.00 14.00 15.00 16.00 17.00 18.00 19.00 20.00 21.00 22.00 23.00 24.00 25.00 26.00 27.00 28.00 29.00 30.00 31.00 32.00 33.00 34.00 35.00

Create two variations of the path-along-x. Calculate the action. Show once again that the Lagrangian-action is indeed smallest for the straight path.

In [8]:
(defn make-varied-path [ε t0 t1]
 (+ path-along-x (* ε (make-η (up #(* 0 %) identity #(* 5.0 (sin %))) t0 t1))))

(def small-varied-path (make-varied-path 0.01 0 10))
(def large-varied-path (make-varied-path 0.02 0 10))

[(Lagrangian-action (lg/L-free-particle 3.0) path-along-x 0.0 10.0)
 (Lagrangian-action (lg/L-free-particle 3.0) small-varied-path 0.0 10.0)
 (Lagrangian-action (lg/L-free-particle 3.0) large-varied-path 0.0 10.0)]

[375.0 383.8260332594603 410.3041330378416]

Create data to plot the three paths in the xz plane along with their actions.

In [9]:
(def path-points-2
    {orange (make-path-txyz path-along-x 0 10 50)
     blue (make-path-txyz small-varied-path 0 10 50)
     red (make-path-txyz large-varied-path 0 10 50)})

#'book-examples/path-points-2

Plot the three paths.

In [10]:
(-> (points-xy->plot path-points-2)
    export-viz
    display/make-html)

<?xml version="1.0"?>
 1.00 2.00 3.00 4.00 5.00 6.00 7.00 8.00 9.00 10.00 11.00 12.00 13.00 14.00 15.00 16.00 17.00 18.00 19.00 20.00 21.00 22.00 23.00 24.00 25.00 26.00 27.00 28.00 29.00 30.00 31.00 32.00 33.00 34.00 35.00 36.00 37.00 38.00 39.00 40.00 41.00 42.00 43.00 44.00 45.00 46.00 47.00 48.00 49.00 50.00 51.00 -2.00 -1.00 0.00

#### Finding trajectories that minimize the action

The SICM library provides a procedure that constructs a one dimensional path (along, say, the z axis) using an interpolation polynomial: `(make-path t0 q0 t1 q1 qs)`, where q0 and q1 are the endpoints, t0 and t1 are the corresponding times, and qs is a list of intermediate points. I give an example (note that the result, `initial-path`, is itself a function):

In [11]:
(def pi-half (* 0.5 Math/PI))
(def initial-qs [0.1 0.2 0.2])
(def initial-path (lg/make-path 0 1.0 pi-half 0.0 initial-qs))

#'book-examples/initial-path

Construct a parametric action that is just the action computed along that parametric path. Find approximate solution paths of a free particle and the harmonic oszillator respectively (hint: use the SICM procedure `multidimensional-minimize`).

In [12]:
(defn parametric-path-actn
  [Lagrangian t0 q0 t1 q1]
  (fn [qs]
    (let [path (lg/make-path t0 q0 t1 q1 qs)]
      (Lagrangian-action Lagrangian path t0 t1))))

(defn fnd-path
  [Lagrangian t0 q0 t1 q1 initial-qs]
  (let [minimizing-qs
        (mn/multidimensional-minimize
          (parametric-path-actn Lagrangian t0 q0 t1 q1)
          initial-qs)]
    (lg/make-path t0 q0 t1 q1 minimizing-qs)))

(def free-path 
  (fnd-path (lg/L-free-particle 3.0) 0.0 1.0 pi-half 0.0 initial-qs))

(def harmonic-path 
  (fnd-path (lg/L-harmonic 1.0 1.0) 0.0 1.0 pi-half 0.0 initial-qs))

#'book-examples/harmonic-path

Make a plot of these one dimensional paths, this time not in the x-z plane but in the t-z plane. This shows that, upon optimization, the initial-path turns into a streight line and a sinusoidal curve respectively.

In [13]:
(defn make-path-tz [fn_t t0 t1 nofsteps]
  (map #(vector % 0 0 (fn_t %)) (alt-range t0 t1 nofsteps)))

(def plot-3
    (let [i (make-path-tz initial-path 0 pi-half 50)
          f (make-path-tz free-path 0 pi-half 50)
          h (make-path-tz harmonic-path 0 pi-half 50)]
      {orange i blue f red h}))

#'book-examples/plot-3

In [14]:
(-> (points-tz->plot plot-3)
    export-viz
    display/make-html)

<?xml version="1.0"?>
 0.00 1.00 0.00 1.00

Show that your numerically attained harmonic-path approximates the well known analytic solution $x(t) = cos(t)$.

In [15]:
(-> (points-tz->plot
     {orange 
      (make-path-tz #(- (cos %) (harmonic-path %)) 0 pi-half 50)})
    export-viz
    display/make-html)

;Here the y-axis lacks tick marks. The graph in fact makes very very small oscillations around zero.
;This plot badly needs improvement at least in that respect.

<?xml version="1.0"?>
 0.00 1.00 0.00

Calculate the Lagrange equation of the harmonic oszillator.

In [16]:
(tex (((Lagrange-equations (lg/L-harmonic 'm 'k)) (literal-function 'q)) 't))

## 1.5 The Euler-Lagrange Equations

### 1.5.2 Computing Lagrange's Equations

#### The free particle

State the dynamic equation of motion (i.e. the Lagrange equation a.k.a Newton's second law) of the free particle.

In [17]:
(tex (((Lagrange-equations (lg/L-free-particle 'm)) (literal-function 'q)) 't))

Check that an arbitrary straight-line path satisfies this equation, i.e. that inserting a straight line for $q(t)$ gives identically zero (strictly speaking the zero covector of three dimensions).

In [18]:
(letfn [(straight-line [t]
  (up (+ (* 'a t) 'a0)
    (+ (* 'b t) 'b0)
    (+ (* 'c t) 'c0)))]
   (tex (((Lagrange-equations (lg/L-free-particle 'm)) straight-line) 't)))

#### The harmonic oscillator

State the dynamic equation of motion for the harmonic oszillator with arbitrary mass and spring constant.

In [19]:
(tex (((Lagrange-equations (lg/L-harmonic 'm 'k)) (literal-function 'q)) 't))

Plug in a sinusoid with arbitrary amplitude $A$, frequency $\omega$ and phase $\phi$ and show that the only solutions allowed are ones where $\omega = \sqrt{k/m}$ 

In [20]:
(letfn [(proposed-solution [t]
  (* 'A (cos (+ (* 'omega t) 'φ))))]
  (tex (((Lagrange-equations (lg/L-harmonic 'm 'k)) proposed-solution) 't)))

#### Exercise 1.11: Kepler's third law

Show that a planet in circular orbit satisfies Kepler's third law $n^2a^3=G(M_1+m_2)$, where $n$ is the angular frequency of the orbit and $a$ is the distance between sun and planet. (Hint: use the reduced mass to construct the Lagrangian)

In [21]:
(defn gravitational-energy [G M_1 m_2]
  (fn [r]
   (- (/ (* G M_1 m_2) r))))

(defn circle [t]
  (up 'a (* 'n t)))

(let [lagrangian (lg/L-central-polar     
                  (/ (* 'M_1 'm_2) (+ 'M_1 'm_2))     
                  (gravitational-energy 'G 'M_1 'm_2))]
      (tex (((Lagrange-equations lagrangian) circle) 't)))

## 1.6 How to find Lagrangians

#### Central force field

State the dynamic equation of motion for the uniform acceleration and the central potential, the latter in rectangular as well as in polar coordinates.

In [22]:
(tex (up
           (((Lagrange-equations
              (lg/L-uniform-acceleration 'm 'g))
            (up (literal-function 'x)
              (literal-function 'y)))
           't)
           
           (((Lagrange-equations
              (lg/L-central-rectangular 'm (literal-function 'U)))
            (up (literal-function 'x)
              (literal-function 'y)))
           't)
           
           (((Lagrange-equations
              (lg/L-central-polar 'm (literal-function 'U)))
            (up (literal-function 'r)
              (literal-function 'phi)))
           't)))

### 1.6.1 Coordinate transformations

Calculate the $[\dot x \space \dot y]$ velocity vector in polar coordinates.

In [23]:
(tex (velocity ((F->C p->r) (->local 't (up 'r 'φ) (up 'rdot 'φdot)))))

Calculate the lagrangian in polar coordinates twice. Once directly and once via the lagrangian in rectangular coordinates.

In [24]:
(defn L-alternate-central-polar
  [m U]
  (compose (lg/L-central-rectangular m U) (F->C p->r)))

(tex
 (let [lcp ((lg/L-central-polar 'm (literal-function 'U))
            (->local 't (up 'r 'φ) (up 'rdot 'φdot)))
       lacp ((L-alternate-central-polar 'm (literal-function 'U))
             (->local 't (up 'r 'φ) (up 'rdot 'φdot)))]
   (up lcp lacp)))

#### Coriolis and centrifugal forces

State, in cartesian coordinates, the Lagrangian for the two dimensional free particle in a rotating coordinate system.

In [25]:
(def L-free-rectangular lg/L-free-particle)

(defn L-free-polar [m]
 (compose (L-free-rectangular m) (F->C p->r)))

(defn F [Omega]
 (fn [[t [r theta]]]
  (up r (+ theta (* Omega t)))))

(defn L-rotating-polar [m Omega]
 (compose (L-free-polar m) (F->C (F Omega))))

(defn r->p
  "rectangular to polar coordinates of state."
  [[_ [x y :as q]]]
  (up (sqrt (square q)) (atan (/ y x))))

(defn L-rotating-rectangular [m Omega]
  (compose (L-rotating-polar m Omega) (F->C r->p)))

(def L-simplify
  (simplify ((L-rotating-rectangular 'm 'Omega)
       	(up 't (up 'x_r 'y_r) (up 'xdot_r 'ydot_r)))))

(tex L-simplify)

Derive the equations of motion, in which the centrifugal and the coriolis force appear. (Hint: take the expression just obtained and use this data as code via Clojure's macro feature).

In [26]:
(defmacro L-fn [args1 args2]
  `(fn ~args1 (fn ~args2 ~L-simplify)))

(def L-rotating-rectangular-simp (L-fn [m Omega] [[t [x_r y_r] [xdot_r ydot_r]]]))

(tex
  (((Lagrange-equations (L-rotating-rectangular-simp 'm 'Omega))
     (up (literal-function 'x_r) (literal-function 'y_r)))
     't))

Plot a clockwise rotating path. (Hints: (1) Use the SICM function "Gamma" to create the triple $(t \: (x \: y) \: (\dot{x} \: \dot{y}))$ which can be transformed, (2) the angular frequency must be negative) 

In [27]:
(defn test-path-2d
  [t]
  (up
   (+ (* 3 t) 7)
   (+ (* 5 t) 11)))

(defn rotate-path [path-fn]
(simplify
 ((F->C p->r)
  ((F->C (F 'Omega))
   ((F->C r->p)
    ((Gamma path-fn) 't))))))

(def xy (nth (rotate-path test-path-2d) 2))
(def x (nth xy 1))
(def y (nth xy 2))

(defmacro P-fn [args1 args2]
  `(fn ~args1 (up (fn ~args2 ~x) (fn ~args2 ~y))))

(def rotating-path-2d (P-fn [Omega] [t]))

(let [NegativeOm -3]
    (->
        (points-xy->plot {orange (make-path-txyz (rotating-path-2d NegativeOm) 0 3 100)})
        export-viz
        display/make-html))

<?xml version="1.0"?>
 -21.00 -20.00 -19.00 -18.00 -17.00 -16.00 -15.00 -14.00 -13.00 -12.00 -11.00 -10.00 -9.00 -8.00 -7.00 -6.00 -5.00 -4.00 -3.00 -2.00 -1.00 0.00 1.00 2.00 3.00 4.00 5.00 6.00 7.00 8.00 9.00 10.00 11.00 12.00 13.00 14.00 15.00 16.00 17.00 18.00 19.00 20.00 21.00 22.00 23.00 24.00 25.00 26.00 27.00 -30.00 -29.00 -28.00 -27.00 -26.00 -25.00 -24.00 -23.00 -22.00 -21.00 -20.00 -19.00 -18.00 -17.00 -16.00 -15.00 -14.00 -13.00 -12.00 -11.00 -10.00 -9.00 -8.00 -7.00 -6.00 -5.00 -4.00 -3.00 -2.00 -1.00 0.00 1.00 2.00 3.00 4.00 5.00 6.00 7.00 8.00 9.00 10.00 11.00 12.00 13.00 14.00 15.00 16.00 17.00 18.00 19.00 20.00 21.00 22.00 23.00 24.00

Show that this path indeed satiesfies the equations of motion in a rotating coordinate system.

In [45]:
(tex
   (let [Om 'Omega
         NegativeOm (* -1 Om)]
     (((Lagrange-equations (L-rotating-rectangular-simp 'm Om))
       (rotating-path-2d NegativeOm))
      't)))

### 1.6.2 Systems with rigid constraints

#### A pendulum driven at the pivot

State Lagrange’s equation for this system.

In [46]:
(defn T-pend
  [m l _ ys]
  (fn [local]
    (let [[t theta thetadot] local
          vys (D ys)]
      (* 1/2 m
         (+ (square (* l thetadot))
            (square (vys t))
            (* 2 l (vys t) thetadot (sin theta)))))))

(defn V-pend
  [m l g ys]
  (fn [local]
    (let [[t theta _] local]
      (* m g (- (ys t) (* l (cos theta)))))))

(def L-pend (- T-pend V-pend))

(def θ (literal-function 'θ))
(def y_s (literal-function 'y_s))

(tex
  (((Lagrange-equations (L-pend 'm 'l 'g y_s)) θ) 't))

State the Lagrangian

In [47]:
(tex
  ((L-pend 'm 'l 'g y_s) (->local 't 'θ 'θdot)))

### 1.6.3 Constraints as Coordinate Transformations

Derive the previous result by using coordinate transformations.

In [48]:
(defn L-uniform-acceleration [m g]
  (fn [[_ [_ y] v]]
    (- (* 1/2 m (square v)) (* m g y))))

(defn dp-coordinates [l y_s]
  (fn [[t θ]]
    (let [x (* l (sin θ))
          y (- (y_s t) (* l (cos θ)))]
      (up x y))))

(defn L-pend2 [m l g y_s]
  (comp (L-uniform-acceleration m g)
        (F->C (dp-coordinates l y_s))))
(tex
        ((L-pend2 'm 'l 'g y_s) (->local 't 'θ 'θdot)))

### 1.8.3 Central Forces in Three Dimensions

Calculate the z-component of the angular momentum of an arbitrary path in rectangular and spherical coordinates.

In [49]:
(def rectangular-state (up 't
                           (up 'x 'y 'z)
                           (up 'xdot 'ydot 'zdot)))

(def spherical-state (up 't
                         (up 'r 'θ 'φ)
                         (up 'rdot 'θdot 'φdot)))

(defn ang-mom-z [m]
  (fn [[_ xyz v]]
    (get (cross-product xyz (* m v)) 2)))

(tex
  (up
    ((ang-mom-z 'm) rectangular-state)
    ((compose (ang-mom-z 'm) (F->C s->r)) spherical-state)))

Using sherical coordinates, calculate the generalized forces and the generalized momenta of a planet moving in a central potential. Thus show that the momentum conjugate to the third coordinate $\phi$ is (1) conserved (because the respective force is zero) and (2) identical the z-component of the angular momentum.

In [50]:
(def V (literal-function 'V))

(defn T3-spherical [m]
 (compose (L-free-rectangular m) (F->C s->r)))

(defn L3-central [m Vr]
  (let [Vs (fn [[_ [r]]] (Vr r))]
    (- (T3-spherical m) Vs)))

(tex
  (up
    (((partial 1) (L3-central 'm V)) spherical-state)
    (((partial 2) (L3-central 'm V)) spherical-state)))

Show that the energy state function computed from the Lagrangian for a central field is in fact T + V.

In [51]:
(tex
  (up  
    ((T3-spherical 'm) (->local 't (up 'r 'θ 'φ) (up 'rdot 'θdot 'φdot)))
    ((Lagrangian->energy (L3-central 'm V)) spherical-state)))